In [1]:
import os
import groq

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
from langchain.tools import tool

In [4]:
import requests
from pydantic.v1 import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [5]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [11]:
tools = [get_current_temperature, search_wikipedia]

In [12]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [16]:
functions = [format_tool_to_openai_function(f) for f in tools]
model = ChatGroq(temperature=0).bind(tools=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}")
])

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [17]:
result = chain.invoke({"input": "what is the weather is Tokyo?"})

BadRequestError: Error code: 400 - {'error': {'message': "'tools.0.type' : property 'type' is missing", 'type': 'invalid_request_error'}}

In [18]:
# place to pass in list of messages
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="urgent_scratchpad")
])

In [19]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [20]:
result1 =  chain.invoke({
    "input": "what is the weather in Kyoto",
    "agent_scratchpad": []
})

KeyError: "Input to ChatPromptTemplate is missing variables {'urgent_scratchpad'}.  Expected: ['input', 'urgent_scratchpad'] Received: ['input', 'agent_scratchpad']"

In [21]:
observation = get_current_temperature(result1.tool_input)
observation

NameError: name 'result1' is not defined

In [22]:
from langchain.agents.format_scratchpad import format_to_openai_functions

result1.message_log

NameError: name 'result1' is not defined

In [23]:
format_to_openai_functions([(result1, observation), ])

NameError: name 'result1' is not defined

In [24]:
result2 = chain.invoke({
    "input": "what is the weather is Osaka?",
    "agent_scratchpad": format_to_openai_functions([(result1, observation), ])
})

NameError: name 'result1' is not defined

In [29]:
from langchain.schema.agent import AgentFinish
def run_agent(user_input):
    intermidiate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input,
            "agent_scratchpad": format_to_openai_functions([(result1, observation), ])
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermidiate_steps.append((result, observation))

In [30]:
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate"]) | chain
)

In [31]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input,
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature
        }[result.tool]
        observation = tool.run(result, tool_input)
        intermediate_steps.append(result, observation)

In [32]:
run_agent("what is the weather in Osaka?")

KeyError: 'intermediate'

In [34]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)
agent_executor.invoke({"input":"What is the temperature in Hokkaido?"})



> Entering new AgentExecutor chain...


KeyError: 'intermediate'

In [35]:
# Add message placeholder for match history

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", {"input"}),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

ValueError: 

In [36]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate"]) | prompt | model | OpenAIFunctionsAgentOutputParser()
)

In [37]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

In [38]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)
agent_executor.invoke({"input": "what is my name?"})

In [39]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in"""
    print(type(query))
    return query[::-1]